# Human Activity Recognition for Healthy Lifestyle Monitoring

Group 40: Alvaro Menéndez Ros, Juan José Leguineche Gállego, Pablo Herrero Ortiz

## Description

This project implements a **Human Activity Recognition (HAR) system** using smartphone sensor data to classify five core daily activities: sitting down, standing up, walking, running, and climbing stairs. The solution follows a complete pipeline, including:

1. **Data Acquisition**: Collecting raw accelerometer, gravitational and gyroscope signals from smartphones during different activities.  
2. **Preprocessing**: Cleaning and segmenting noisy time-series data into structured windows.  
3. **Feature Extraction**: Deriving meaningful statistical and frequency-based features that capture movement patterns.  
4. **Model Development**: Training supervised machine learning models to distinguish between activities.  
5. **Evaluation**: Assessing model performance using accuracy, precision, recall, and confusion matrices to ensure robustness.  

By simulating real-world challenges in digital health, the system demonstrates how wearable sensor data can be transformed into actionable insights for **health monitoring and activity tracking applications**.  


## Data Acquisition



 ## Preprocessing


## Feature extraction

## Model development

## Evaluation